# Eternity II Tutorial

The goal of [Eternity II](https://en.wikipedia.org/wiki/Eternity_II_puzzle) is to place triangular pieces on a board such that squares of the same color are formed. In the full game, the board is a 16x16 grid, and 22 colors. The goal is to place all 256 pieces on the board such that all triangles are formed. The game was named eternity II because there used to be a 2 million $ prize for first player to solve the puzzle. The prize has been removed since, but the puzzled has never been solved to this day. Here is an example of a solved board (4x4 instance):

![Solved 4x4 Board](img/EternityII.png)

In this tutorial, we will aim to solve smaller instances of Eternity II with SeaPearl. 

## Setup

We will begin by activating the environment and importing the necessary packages.

In [98]:
using Revise
using Pkg
Pkg.activate("../../../")
using SeaPearl

  Activating project at `c:\Users\leobo\Desktop\École\Poly\SeaPearl\SeaPearlZoo.jl`


## Problem Formulation

Instances are located in the "./data" directory. We need to load them into a Julia variable. We will use the 3x3 instance as an example. The file is a text file with the following format:
```
3 3
0 0 1 1
0 0 1 2
0 0 2 1
0 0 2 2
0 1 3 2
0 1 4 1
0 2 3 1
0 2 4 2
3 3 4 4
```
The first line contains the dimensions of the board. The following lines contain the pieces. Each line contains 4 numbers that represent up, down, left and right colors of the piece. In the 3x3 example, colors range from 0 to 4. We will build simple utilities to read instances.

In [99]:

"""EternityInputData(n::Int, m::Int, pieces::Matrix{Int})

# Arguments
- 'n::Int' : number of rows
- 'm::Int' : number of columns
- 'pieces::Matrix{Int}' : pieces of the puzzle (n * m, 4) with the colors
"""
struct EternityInputData
    n::Int
    m::Int
    pieces::Matrix{Int}
end

"""
parseEternityInput(filename::String; order=[1,2,3,4])

Parse an instance file and load it into a EternityInputData struct.

# Arguments
- 'filename::String' : path to the instance file
- order=[1,2,3,4] : order of the colors in the file
"""
function parseEternityInput(filename::String; order=[1,2,3,4])
    raw_input = nothing
    open(filename, "r") do openedFile
        raw_input = read(openedFile, String)
    end
    lines = split(raw_input, '\n')
    firstLine = split(lines[1], ' ')
    n = parse(Int, firstLine[1])
    m = parse(Int, firstLine[2])
    pieces = Matrix{Int}(undef, n*m, 4)

    for i = 2:n*m+1
        line = split(lines[i],' ')
        for j =1:4
            pieces[i-1,j] = parse(Int, line[order[j]])
        end
    end

    return EternityInputData(n, m, pieces)
end

parseEternityInput

In [100]:
eternityInput::EternityInputData = parseEternityInput("./data/eternity3x3")

EternityInputData(3, 3, [0 0 1 1; 0 0 1 2; … ; 0 2 4 2; 3 3 4 4])

## Modeling the Problem, pt. 1 - Simple Model

We will now build a simple model for the problem.

In [101]:
trailer = SeaPearl.Trailer()
model = SeaPearl.CPModel(trailer)
model.limit.numberOfSolutions = nothing

n = eternityInput.n
m = eternityInput.m
pieces = eternityInput.pieces
num_colors = maximum(pieces)

table = Matrix{Int}(undef, 5, 4 * n * m)

for tile = 1: n * m
    for color = 1:4
        table[1, 4 * (tile - 1) + color] = tile
        for j = 2:5
            table[j, 4 * (tile - 1) + color] = pieces[tile, (j + color + 1) % 4 + 1]
        end
    end
end

id = Matrix{SeaPearl.AbstractIntVar}(undef, n, m) # id of the tile
up = Matrix{SeaPearl.AbstractIntVar}(undef, n, m) # up
right = Matrix{SeaPearl.AbstractIntVar}(undef, n, m) # right
down = Matrix{SeaPearl.AbstractIntVar}(undef, n, m) # down
left = Matrix{SeaPearl.AbstractIntVar}(undef, n, m) # left

for i = 1:n, j=1:m
    # Add ID variables
    id[i,j] = SeaPearl.IntVar(1, n * m, "id_"*string(i)*string(j), trailer)
    SeaPearl.addVariable!(model, id[i,j]; branchable=true)
    # Add up variables
    up[i, j] = SeaPearl.IntVar(0, num_colors, "u_" * string(i) * string(j), trailer)
    SeaPearl.addVariable!(model, up[i,j]; branchable=false)
    # Add right variables
    right[i, j] = SeaPearl.IntVar(0, num_colors, "r_" * string(i) * string(j), trailer)
    SeaPearl.addVariable!(model, right[i,j]; branchable=false)
    # Add down variables
    down[i, j] = SeaPearl.IntVar(0, num_colors, "d_" * string(i) * string(j), trailer)
    SeaPearl.addVariable!(model, down[i,j]; branchable=false)
    # Add left variables
    left[i, j] = SeaPearl.IntVar(0, num_colors, "l_" * string(i) * string(j), trailer)
    SeaPearl.addVariable!(model, left[i,j]; branchable=false)

    vars = SeaPearl.AbstractIntVar[id[i,j], up[i,j], right[i,j],down[i,j], left[i,j]]
    push!(model.constraints, SeaPearl.TableConstraint(vars, table, trailer))

    if (j==m) push!(model.constraints, SeaPearl.EqualConstant(right[i,j], 0, trailer)) end
    if (j==1) push!(model.constraints, SeaPearl.EqualConstant(left[i,j], 0, trailer)) end
    if (i==1) push!(model.constraints, SeaPearl.EqualConstant(up[i,j], 0, trailer)) end
    if (i==n) push!(model.constraints, SeaPearl.EqualConstant(down[i,j], 0, trailer)) end
end

for i = 1:n, j=1:m
    if (j < m) push!(model.constraints, SeaPearl.Equal(right[i,j], left[i,j+1], trailer)) end
    if (i < n) push!(model.constraints, SeaPearl.Equal(down[i,j], up[i+1,j], trailer)) end
end

push!(model.constraints, SeaPearl.AllDifferent(id, trailer))
model.adhocInfo = Dict([("n", n), ("m", m)])


Dict{String, Int64} with 2 entries:
  "m" => 3
  "n" => 3

## Solving the problem

Now that we have a model for the 3x3 eternity, we can begin to solve it.

In [102]:
variableSelection = SeaPearl.MinDomainVariableSelection{false}()
valueSelection = SeaPearl.BasicHeuristic()

status = @time SeaPearl.solve!(model; variableHeuristic=variableSelection, valueSelection=valueSelection)

 20.044618 seconds (20.43 M allocations: 1.046 GiB, 1.14% gc time, 99.78% compilation time)


:Optimal

## Plotting the solution

We can see that the problem was solved to optimality. Now let's visualize the results we obtained.

In [121]:
function outputFromSeaPearl(model::SeaPearl.CPModel)
    solutions = model.statistics.solutions
    nb_sols = length(solutions)
    n = model.adhocInfo["n"]
    m = model.adhocInfo["m"]
    orientation = Array{Int,4}(undef, nb_sols, n, m, 5)
    for (ind, sol) in enumerate(solutions)
        for i in 1:n, j in 1:m
            orientation[ind, i, j, 1] = sol["id_"*string(i)*string(j)]
            orientation[ind, i, j, 2] = sol["u_"*string(i)*string(j)]
            orientation[ind, i, j, 3] = sol["r_"*string(i)*string(j)]
            orientation[ind, i, j, 4] = sol["d_"*string(i)*string(j)]
            orientation[ind, i, j, 5] = sol["l_"*string(i)*string(j)]
        end
    end
    return OutputDataEternityII(nb_sols, orientation)
end

struct OutputDataEternityII
    nb_sols::Int
    orientation::Array{Int,4} # dims = (nb_sols, n, m, 5) where five corresponds to (id, u,r, d, l) (u=upper edge, ...)
end

"""
    print_eternity2(sol::Array{Int,4})

print a solution

# Arguments
- 'sol::Array{Int, 4}' : one solution from OutputDataEternityII
"""
function print_eternity2(sol::Array{Int,4})
    id = sol[:,:,1]
    u = sol[:,:,2]
    r = sol[:,:,3]
    d = sol[:,:,4]
    l = sol[:,:,5]
    n = size(id,1)
    m = size(id,2)
    print(" ")
    for k in 1:9*m
        print("-")
    end
    println()
    for i in 1:n
        print("|")
        for j in 1:m
            printstyled("   "*string(u[i,j],pad=2)*"   ", color=u[i,j])
            print("|")
        end
        println()
        print("|")
        for j in 1:m
            printstyled(string(l[i,j],pad=2),color=l[i,j])
            printstyled(" "*string(id[i,j],pad=2)*" ")
            printstyled(string(r[i,j],pad=2),color=r[i,j])
            print("|")
        end
        println()
        print("|")
        for j in 1:m
            printstyled("   "*string(d[i,j],pad=2)*"   ", color=d[i,j])
            print("|")
        end
        println()
        print(" ")
        for k in 1:9*m
            print("-")
        end
        println()
    end
end

print_eternity2

In [124]:
output_data = outputFromSeaPearl(model)

print_eternity2(output_data.orientation[1,:,:,:])

---------------------------
|   00   |   00   |   00   |
|00 02 01|01 06 01|01 03 00|
|   02   |   04   |   02   |
---------------------------
|   02   |   04   |   02   |
|00 07 03|03 09 04|04 08 00|
|   01   |   03   |   02   |
---------------------------
|   01   |   03   |   02   |
|00 01 01|01 05 02|02 04 00|
|   00   |   00   |   00   |
---------------------------
